In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import pickle
from sklearn.ensemble import AdaBoostClassifier
import time

In [2]:
tf.test.is_gpu_available(cuda_only=False,min_cuda_compute_capability=None)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [6]:
#X = pd.read_csv('Data_25_-2.0_-2.0_6.0.csv').values
#y = pd.read_csv('Data_25_-2.0_-2.0_6.0_y.csv').values
X = pd.read_csv('preprocessed_data/Data_25_2.0_-1.0_1.0.csv').values
y = pd.read_csv('preprocessed_data/Data_25_2.0_-1.0_1.0_y.csv').values



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
print(X.shape)
print(y.shape)
print('zeros', np.count_nonzero(y == 0))
print('ones', np.count_nonzero(y == 1))
print('north', np.count_nonzero(X[:,7] == 1))
print('south', np.count_nonzero(X[:,8] == 1))
print('east', np.count_nonzero(X[:,9] == 1))
print('west', np.count_nonzero(X[:,10] == 1))

(31866, 11)
(31866, 1)
zeros 5788
ones 26078
north 13661
south 12005
east 0
west 6200


## Deep neural network

In [4]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_dim=11),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='softmax')
])

batch_size = 5000
epochs = 100
s_time= time.time()
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

#X_train_sub = X_train[:, 6:]
#X_test_sub = X_test[:, 6:]

history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_test, y_test))

score = model.evaluate(X_test, y_test, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])
print('Time', time.time()-s_time)

Train on 25492 samples, validate on 6374 samples
Epoch 1/100
25492/25492 [==============================] - 1s 52us/sample - loss: 0.3958 - accuracy: 0.3986 - val_loss: 0.1192 - val_accuracy: 0.8549
Epoch 2/100
25492/25492 [==============================] - 0s 2us/sample - loss: 0.1218 - accuracy: 0.8536 - val_loss: 0.1186 - val_accuracy: 0.8605
Epoch 3/100
25492/25492 [==============================] - 0s 1us/sample - loss: 0.1084 - accuracy: 0.8678 - val_loss: 0.0822 - val_accuracy: 0.8839
Epoch 4/100
25492/25492 [==============================] - 0s 1us/sample - loss: 0.0687 - accuracy: 0.9020 - val_loss: 0.0641 - val_accuracy: 0.9167
Epoch 5/100
25492/25492 [==============================] - 0s 1us/sample - loss: 0.0658 - accuracy: 0.9082 - val_loss: 0.0521 - val_accuracy: 0.9270
Epoch 6/100
25492/25492 [==============================] - 0s 1us/sample - loss: 0.0572 - accuracy: 0.9189 - val_loss: 0.0529 - val_accuracy: 0.9233
Epoch 7/100
25492/25492 [==============================]

In [22]:
# using speed and origin only
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_dim=6),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='softmax')
])

batch_size = 5000
epochs = 50
s_time= time.time()
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

X_train_sub = X_train[:, :6]
X_test_sub = X_test[:, :6]

history = model.fit(X_train_sub, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_test_sub, y_test))

score = model.evaluate(X_test_sub, y_test, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])
print('Time', time.time()-s_time)
model.save('models/model_-2_6.h5')


Train on 36777 samples, validate on 9195 samples
Epoch 1/50
36777/36777 [==============================] - 1s 18us/sample - loss: 0.1775 - accuracy: 0.7581 - val_loss: 0.1032 - val_accuracy: 0.8895
Epoch 2/50
36777/36777 [==============================] - 0s 1us/sample - loss: 0.0920 - accuracy: 0.8962 - val_loss: 0.0762 - val_accuracy: 0.9143
Epoch 3/50
36777/36777 [==============================] - 0s 1us/sample - loss: 0.0667 - accuracy: 0.9217 - val_loss: 0.0532 - val_accuracy: 0.9485
Epoch 4/50
36777/36777 [==============================] - 0s 1us/sample - loss: 0.0486 - accuracy: 0.9541 - val_loss: 0.0407 - val_accuracy: 0.9610
Epoch 5/50
36777/36777 [==============================] - 0s 1us/sample - loss: 0.0390 - accuracy: 0.9590 - val_loss: 0.0344 - val_accuracy: 0.9638
Epoch 6/50
36777/36777 [==============================] - 0s 1us/sample - loss: 0.0337 - accuracy: 0.9620 - val_loss: 0.0310 - val_accuracy: 0.9651
Epoch 7/50
36777/36777 [==============================] - 0s 1

In [7]:
from tensorflow.keras.models import load_model
import numpy as np
model1 = load_model('models/model_-2_6.h5')
x1 = np.array([[0,2,3,0,5,6]])
o=model1.predict(x1)
print('predict',o[0][0])

predict 0.9997602


In [31]:
y_out=model.predict(X_test)
for i in range(len(y_out)):
    print(y_test[i],y_out[i])

[0] [0.00184283]
[1] [0.99999946]
[0] [2.115965e-06]
[0] [1.2516975e-06]
[0] [0.00027665]
[0] [0.]
[1] [0.9998117]
[0] [2.7120113e-06]
[1] [0.999998]
[0] [1.9073486e-06]
[0] [3.3825636e-05]
[0] [2.2143126e-05]
[0] [9.268522e-06]
[1] [0.9999868]
[1] [0.99999976]
[0] [7.1525574e-07]
[0] [0.]
[0] [5.9604645e-08]
[0] [6.467104e-06]
[0] [3.993511e-06]
[0] [2.9802322e-08]
[0] [1.013279e-06]
[0] [8.940697e-08]
[1] [0.99999905]
[0] [1.1920929e-07]
[0] [0.]
[0] [2.9802322e-08]
[0] [3.0398369e-06]
[0] [0.]
[0] [0.]
[1] [0.99999976]
[1] [0.99999976]
[0] [3.2782555e-07]
[0] [8.60095e-05]
[1] [0.9999998]
[0] [0.]
[1] [0.9999974]
[1] [0.99999547]
[0] [0.]
[0] [4.9322844e-05]
[1] [0.9999979]
[0] [2.9802322e-08]
[0] [1.013279e-06]
[1] [0.9999983]
[1] [0.9999999]
[0] [1.1920929e-07]
[0] [5.6028366e-06]
[0] [0.00010598]
[0] [1.4901161e-06]
[0] [1.8715858e-05]
[0] [0.00036451]
[1] [0.99999976]
[1] [0.99999976]
[0] [1.6361475e-05]
[0] [7.122755e-06]
[1] [0.99999946]
[1] [0.9999999]
[0] [0.00021872]
[0] [0

## Random forest

In [7]:
def accuracy(predictions, y):
    errors = abs(predictions - y)
    # Display the performance metrics
    print('Mean Absolute Error:', round(np.mean(errors), 2)**2, 'degrees.')
    mape = np.mean(100 * (errors / len(y)))
    accuracy = 100 - mape
    print('Accuracy:', round(accuracy, 2), '%.')

In [ ]:
model = RandomForestClassifier(n_estimators=1000, 
                               max_features = None,
                               n_jobs=10, verbose = 1)
s_time= time.time()
# Fit on training data
model.fit(X_train, y_train)

n_nodes = []
max_depths = []
# Stats about the trees in random forest
for ind_tree in model.estimators_:
    n_nodes.append(ind_tree.tree_.node_count)
    max_depths.append(ind_tree.tree_.max_depth)
    
print(f'Average number of nodes {int(np.mean(n_nodes))}')
print(f'Average maximum depth {int(np.mean(max_depths))}')

train_predictions = model.predict(X_train)
#train_rf_probs = model.predict_proba(X_train)[:, 1]
test_predictions = model.predict(X_test)
#rf_probs = model.predict_proba(X_test)[:, 1]


print('train accuracy:')
accuracy(train_predictions, y_train)

print('test accuracy:')
accuracy(test_predictions, y_test)


print('Time', time.time()-s_time)

C:\Users\rashid\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:    1.2s
[Parallel(n_jobs=10)]: Done 180 tasks      | elapsed:    7.6s
[Parallel(n_jobs=10)]: Done 430 tasks      | elapsed:   17.9s


## XG Boost

In [11]:
X_train_sub = X_train[:, :6]
X_test_sub = X_test[:, :6]

dtrain = xgb.DMatrix(X_train_sub, label=y_train)
dtest = xgb.DMatrix(X_test_sub, label=y_test)

param = {'max_depth': 2, 'eta': 1, 'objective': 'binary:logistic'}
param['nthread'] = 4
param['eval_metric'] = 'auc'
evallist = [(dtest, 'eval'), (dtrain, 'train')]

s_time= time.time()
num_round = 100
bst = xgb.train(param, dtrain, num_round, evallist)

ypred_train = bst.predict(dtrain)
ypred_test = bst.predict(dtest)

print('train accuracy:')
#accuracy(ypred_train, y_train)
pickle.dump( {'xg':bst}, open( "models/xg_-1_1_25.p", "wb" ) )

print('test accuracy:')
#accuracy(ypred_test, y_test)

print('Time', time.time()-s_time)

[0]	eval-auc:0.88627	train-auc:0.87657
[1]	eval-auc:0.94388	train-auc:0.93956
[2]	eval-auc:0.96211	train-auc:0.95876
[3]	eval-auc:0.96767	train-auc:0.96410
[4]	eval-auc:0.96984	train-auc:0.96759
[5]	eval-auc:0.97106	train-auc:0.96861
[6]	eval-auc:0.97104	train-auc:0.96962
[7]	eval-auc:0.97152	train-auc:0.97012
[8]	eval-auc:0.97214	train-auc:0.97101
[9]	eval-auc:0.97212	train-auc:0.97179
[10]	eval-auc:0.97337	train-auc:0.97297
[11]	eval-auc:0.97367	train-auc:0.97344
[12]	eval-auc:0.97416	train-auc:0.97410
[13]	eval-auc:0.97416	train-auc:0.97428
[14]	eval-auc:0.97423	train-auc:0.97462
[15]	eval-auc:0.97447	train-auc:0.97481
[16]	eval-auc:0.97446	train-auc:0.97507
[17]	eval-auc:0.97464	train-auc:0.97535
[18]	eval-auc:0.97444	train-auc:0.97548
[19]	eval-auc:0.97447	train-auc:0.97581
[20]	eval-auc:0.97451	train-auc:0.97598
[21]	eval-auc:0.97431	train-auc:0.97603
[22]	eval-auc:0.97449	train-auc:0.97623
[23]	eval-auc:0.97558	train-auc:0.97675
[24]	eval-auc:0.97558	train-auc:0.97705
[25]	eval-

In [9]:
bst.predict(xgb.DMatrix(np.array([[1,2,3,4,5,6]])))[0]

0.0054679625

## Ada Boost

In [67]:
s_time= time.time()
clf = AdaBoostClassifier(n_estimators=500, random_state=0)
clf.fit(X_train, y_train)

print('train accuracy:', clf.score(X_train, y_train))

print('test accuracy:', clf.score(X_test, y_test))
accuracy(clf.predict(X_test), y_test)
print('Time', time.time()-s_time)

C:\Users\rashid\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


train accuracy: 1.0
test accuracy: 0.9892761394101877
Mean Absolute Error: 0.18489999999999998 degrees.
Accuracy: 99.88 %.
Time 1.2909202575683594
